In [53]:
import pandas as pd
import numpy as np

# df=pd.read_csv('./SMSSpamCollection',sep='\t', header=None,na_filter=False,names=['class','text'])

# df['class'] = df["class"].astype('category')

# df['true_label'] = df['class'].cat.codes

# np.random.seed(1234)
# msk = np.random.rand(len(df)) < 0.66

# train_df = df[msk]
# test_df = df[~msk]

# true_labels = test_df['true_label'].tolist()

# train_df.to_pickle("train_df")
# test_df.to_pickle("test_df")

In [156]:
def avglen(x):
    s = 0
    for i,y in x.iteritems():
        s = s+ len(y)
    a = s/x.size    
    return a

def caplen(x):
    s = 0
    for i,y in x.iteritems():
        s = s+ sum(1 for c in y if c.isupper())
    a = s/x.size    
    return a

def capmax(x):
    s = 0
    for i,y in x.iteritems():
        s = max(s, sum(1 for c in y if c.isupper())) 
    return s

def maxlen(x):
    l = 0
    for i,y in x.iteritems():
        l = max(l,len(y))
    return l

def minlen(x):
    l = 5000
    for i,y in x.iteritems():
        l = min(l,len(y))
    return l

train_df.groupby("class").agg({"text": [avglen,maxlen,minlen,caplen,capmax]})

text                                
           avglen maxlen minlen     caplen capmax
class                                            
ham     70.206076    910      2   4.115252    129
spam   140.305720    223     13  15.114398    128

In [200]:
sc = 0
hc = 0
lt = 6
# for v in df.iterrows():
# #     print(v[0])
#     if(v[1]['class']=='spam' and len(v[1]['text'])>lt):
#         sc = sc+1
#     if(v[1]['class']=='ham' and len(v[1]['text'])>lt):
#         hc = hc+1
        
for v in df.iterrows():
#     print(v[0])
    if(v[1]['class']=='spam' and sum(1 for c in v[1]['text'] if c.isupper())>lt):
        sc = sc+1
    if(v[1]['class']=='ham' and sum(1 for c in v[1]['text'] if c.isupper())>lt):
        hc = hc+1
    
print(sc,hc)

591 431


In [40]:
train_df = pd.read_pickle("train_df")
# train_df.head()
train_df.groupby("class").agg({"text": np.count_nonzero})

,text
class,
ham,3193
spam,507


In [19]:
test_df = pd.read_pickle("test_df")

true_labels = test_df['true_label'].tolist()

test_df.groupby("class").agg({"text": np.count_nonzero})


,text
class,
ham,1632
spam,240


In [171]:
from gensim.parsing.preprocessing import STOPWORDS
import gensim.matutils as gm

from gensim.models.keyedvectors import KeyedVectors

# Load pretrained model (since intermediate data is not included, the model cannot be refined with additional data)
model = KeyedVectors.load_word2vec_format('../../../snorkel/tutorials/glove_w2v.txt', binary=False)  # C binary format


wordvec_unavailable= set()
def write_to_file(wordvec_unavailable):
    with open("wordvec_unavailable.txt","w") as f:
        for word in wordvec_unavailable:
            f.write(word+"\n")

def preprocess(tokens):
    btw_words = [word for word in tokens if word not in STOPWORDS]
    btw_words = [word for word in btw_words if word.isalpha()]
    return btw_words

def get_word_vectors(btw_words): # returns vector of embeddings of words
    word_vectors= []
    for word in btw_words:
        try:
            word_v = np.array(model[word])
            word_v = word_v.reshape(len(word_v),1)
            #print(word_v.shape)
            word_vectors.append(model[word])
        except:
            wordvec_unavailable.add(word)
    return word_vectors

def get_similarity(word_vectors,target_word): # sent(list of word vecs) to word similarity
    similarity = 0
    target_word_vector = 0
    try:
        target_word_vector = model[target_word]
    except:
        wordvec_unavailable.add(target_word+" t")
        return similarity
    target_word_sparse = gm.any2sparse(target_word_vector,eps=1e-09)
    for wv in word_vectors:
        wv_sparse = gm.any2sparse(wv, eps=1e-09)
        similarity = max(similarity,gm.cossim(wv_sparse,target_word_sparse))
    return similarity


In [203]:
for v in (train_df.loc[train_df['class'] =='ham']).iterrows():
    print(v[0])
#     print(v[1]['category_name'])
    print(v[1]['text'])

0
Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
1
Ok lar... Joking wif u oni...
6
Even my brother is not like to speak with me. They treat me like aids patent.
10
I'm gonna be home soon and i don't want to talk about this stuff anymore tonight, k? I've cried enough today.
14
I HAVE A DATE ON SUNDAY WITH WILL!!
16
Oh k...i'm watching here:)
17
Eh u remember how 2 spell his name... Yes i did. He v naughty make until i v wet.
20
Is that seriously how you spell his name?
21
I‘m going to try for 2 months ha ha only joking
22
So ü pay first lar... Then when is da stock comin...
23
Aft i finish my lunch then i go str down lor. Ard 3 smth lor. U finish ur lunch already?
25
Just forced myself to eat a slice. I'm really not hungry tho. This sucks. Mark is getting worried. He knows I'm sick when I turn down pizza. Lol
26
Lol your always so convincing.
28
I'm back &amp; we're packing the car now, I'll let you know if there's room
29

In [105]:
def read_words(words_file):
    return [line for line in open(words_file, 'r') ]

words = read_words("blacklist.txt")

spam = [word.strip('-_\n/') for word in words]
spam = [word.replace('-',' ') for word in spam]
print(len(spam))


# l1 = []
# l2 = []
# l3 = []
# for i,w in enumerate(spam):
#     if(i%3==0):
#         l1.append(w)
#     elif(i%3==1):
#         l2.append(w)
#     else:
#         l3.append(w)
        
l1 = set()
l2 = set()
l3 = set()
for i,w in enumerate(spam):
    if(i%3==0):
        l1.add(w)
    elif(i%3==1):
        l2.add(w)
    else:
        l3.add(w)
print(len(l1),len(l2),len(l3))

28294
8846 8879 8861


In [182]:
print(l1)

{'', 'mbt fora', 'packersjers', 'favourable single', 'japanese mont', 'abercrombie deutsch', '6.su', 'jordan sale', '.credopa.in', 'celine prezzi', 'herbaltincture', 'loan.co', 'wprobot', 'website..', 'laminin lpgn', 'teen cam', 'c http', 'replicachina', 'iphone/iphone', 'yourdick', 'sexcam', 'oxycontin', 'foolproof trick', '3.in', 'tattootip', 'chinascarf', 'l.uk.e.w.a.rm', 'coast petite', 'favourite justif', 'salemiami', 'sprawdzonenarzedzia', 'сети работ', 'chinesecheap', 'lululemon loca', 'porn.model', 'fifa', 'actual submit', 'jordan femme', 'belstaffcuero', 'occhiali', 'bioactivity', '.filmi', 'mac cosmetic', 'get professional', 'image/cache', 'my online', 'hollister tiendas', 'lesbos.', 'bcbg casual', 'bulgari brand', '00mg buy', 'celine paris', 'mainslancel', 'unknown.ro', 'baguebulgari', 'officialmailsite', 'cool t shirts', 'belstaff hand', 'replicaleger', 'toile vanessa', 'ženys run', 'lady porn', 'sac reutilis', 'vinter jakker', 'gmailprox', 'allimage', 'bluecap turbo', 'mat

In [263]:
####### Discrete ##########


# Helper function to get last name

import re
def ltp(x):
    return '(' + '|'.join(x) + ')'

# def l1words(c):
#     return (1,1) if re.search(ltp(l1), c['text'], flags=re.I) else (0,0)

# def l2words(c):
#      return (1,1) if re.search(ltp(l2), c['text'], flags=re.I) else (0,0)

# def l3words(c):
#      return (1,1) if re.search(ltp(l3), c['text'], flags=re.I) else (0,0)

# notFree = ['you','toll','your','call','meet','talk','freez']

# def notFreeSpam(c):
#     return (-1,1) if re.search('(free.*'+ltp(notFree)+')|('+ltp(notFree)+'.*free)',\
#                                flags=re.I) else (0,0)

def l1words(c):
    return (1,1) if len(l1.intersection(c['text'].split())) > 0 else (0,0)

def l2words(c):
    return (1,1) if len(l2.intersection(c['text'].split())) > 0 else (0,0)

def l3words(c):
    return (1,1) if len(l3.intersection(c['text'].split())) > 0 else (0,0)



notFree1 = {'toll','Toll','freely','call','meet','talk','feedback'}

def notFreeSpam(c):
    return (-1,1) if 'free' in c['text'].split() and len(notFree.intersection(c['text'].split()))>0 else (0,0)        

notFree2 = {'not free','you are','when','wen'}

def notFreeSpam2(c):
    return (-1,1) if 'free' in c['text'].split() and re.search(ltp(notFree2),c['text'], flags= re.I) else (0,0)        

person1 = {'I','i','u','you','ur','your','our','we','us','you\'re,'}

person2 = {'He','he','She','she','they','They','Them','them','their','Their'}

def personWords(c):
    return (-1,1) if 'free' in c['text'].split() and len(person1.intersection(c['text'].split()))>0 else (0,0)        

def secondPersonWords(c):
    return (-1,1) if 'free' in c['text'].split() and len(person2.intersection(c['text'].split()))>0 else (0,0)        

def noOfCapChars(c):
    return (1,1) if (sum(1 for ch in c['text'] if ch.isupper()) > 6) else (0,0)

LFs = [
   l1words,l2words,l3words,noOfCapChars,notFreeSpam,notFreeSpam2,personWords,secondPersonWords
]

LF_l = [1,1,1,1,-1,-1,-1,-1]


print(len(LFs),len(LF_l))

8 8


In [11]:
##### Continuous ################



def l1words(c):
    sc = 0
    word_vectors = get_word_vectors(c['text'].split())
    l1 = ['free','credit','cheap','apply','buy','attention','shop','sex','soon','now','spam']
    for w in l1:
        sc=max(sc,get_similarity(word_vectors,w))
    return (1,sc)

def l2words(c):
    sc = 0
    l2 = ['gift','click','new','online','discount','earn','miss','hesitate','exclusive','urgent']
    word_vectors = get_word_vectors(c['text'].split())
    for w in l2:
        sc=max(sc,get_similarity(word_vectors,w))
    return (1,sc)

def l3words(c):
    sc = 0
    l3 = ['cash','refund','insurance','money','guaranteed','save','win','teen','weight','hair']
    word_vectors = get_word_vectors(c['text'].split())
    for w in l3:
        sc=max(sc,get_similarity(word_vectors,w))
    return (1,sc)
   
def notFreeSpam(c):
    sc = 0
    notFree = ['not','when','call','meet','talk','feedback','toll']
    word_vectors = get_word_vectors(c['text'].split())
    for w in notFree:
        sc=max(sc,get_similarity(word_vectors,w))
    return (1,sc)
   

def notFreeSpam2(c):
    sc = 0
    notFree2 =  ['not free','you are','when']
    word_vectors = get_word_vectors(c['text'].split())
    for w in notFree2:
        sc=max(sc,get_similarity(word_vectors,w))
    return (1,sc)

def personWords(c):
    sc = 0
    notFree2 = ['I','you','your','we','us']
    word_vectors = get_word_vectors(c['text'].split())
    for w in person1:
        sc=max(sc,get_similarity(word_vectors,w))
    return (-1,sc)
    return (-1,1) if 'free' in c['text'].split() and re.search(ltp(notFree2),c['text'], flags= re.I) else (0,0)        

def secondPersonWords(c):
    sc = 0
    notFree2 = ['he','she','they','them','their']
    word_vectors = get_word_vectors(c['text'].split())
    for w in person1:
        sc=max(sc,get_similarity(word_vectors,w))
    return (-1,sc)
   
    
def noOfCapChars(c):
    l = sum(1 for ch in c['text'] if ch.isupper()) 
    return (1,l/150)

LFs = [
   l1words,l2words,l3words,noOfCapChars,notFreeSpam,notFreeSpam2,personWords,secondPersonWords
]

LF_l = [1,1,1,1,-1,-1,-1,-1]

print(len(LFs),len(LF_l))

8 8


In [254]:
''' output:

    [[[L_x1],[S_x1]],
     [[L_x2],[S_x2]],
     ......
     ......
    ]

'''


def get_L_S_Tensor(df,msg):     
    L_S = []
    print('labelling ',msg,' data')
    for i in range(len(df.index)):
        L_S_ci=[]
        L=[]
        S=[]
        P_ik = []
        for LF in LFs:
#             print(i,LF.__name__)    
#             print(df.iloc[i]['text'])
            l,s = LF(df.iloc[i])
            L.append(l)
            S.append((s+1)/2)  #to scale scores in [0,1] 
        L_S_ci.append(L)
        L_S_ci.append(S)
        L_S.append(L_S_ci) 
        if(i%250==0 and i!=0):
            print(str(i)+'data points labelled in',(time.time() - start_time)/60,'mins')
        
    return L_S


In [264]:
# import matplotlib.pyplot as plt
import time
import numpy as np
start_time = time.time()

lt = time.localtime()

print("started at: {}-{}-{}, {}:{}:{}".format(lt.tm_mday,lt.tm_mon,lt.tm_year,lt.tm_hour,lt.tm_min,lt.tm_sec))


test_L_S = get_L_S_Tensor(test_df,'regex test')
np.save("test_L_S_discrete",np.array(test_L_S))

train_L_S = get_L_S_Tensor(train_df,'regex train')
np.save("train_L_S_discrete",np.array(train_L_S))



# test_L_S = get_L_S_Tensor(test_df,'regex test')
# np.save("test_L_S_smooth",np.array(test_L_S))

# train_L_S = get_L_S_Tensor(train_df,'regex train')
# np.save("train_L_S_smooth",np.array(train_L_S))



print("--- %s seconds ---" % (time.time() - start_time))

# test_L_S = get_L_S_Tensor(test_cands)
# pkl.dump(test_L_S,open("test_L_S.p","wb"))

started at: 9-6-2018, 20:24:46
labelling  regex test  data
250data points labelled in 0.004436842600504557 mins
500data points labelled in 0.008656620979309082 mins
750data points labelled in 0.01289513111114502 mins
1000data points labelled in 0.017128411928812662 mins
1250data points labelled in 0.021420172850290933 mins
1500data points labelled in 0.02580949068069458 mins
1750data points labelled in 0.0302552858988444 mins
labelling  regex train  data
250data points labelled in 0.03731296062469482 mins
500data points labelled in 0.041551840305328366 mins
750data points labelled in 0.045717823505401614 mins
1000data points labelled in 0.04994643131891886 mins
1250data points labelled in 0.054119853178660075 mins
1500data points labelled in 0.05829683542251587 mins
1750data points labelled in 0.06246569554011027 mins
2000data points labelled in 0.06664372682571411 mins
2250data points labelled in 0.07073365052541097 mins
2500data points labelled in 0.0749161958694458 mins
2750data poi

In [311]:
LF_l = [1,1,1,1,-1,-1,-1,-1]

def merge(a,b):
    c = []
    for i in range(len(a)):
        ci = []
        ci_l = a[i,0,:].tolist()+b[i,0,:].tolist()
        ci_s = a[i,1,:].tolist()+b[i,1,:].tolist()
        ci.append(ci_l)
        ci.append(ci_s)
        c.append(ci)
    return c
import numpy as np
test_L_S_s = np.load("test_L_S_smooth.npy")
train_L_S_s = np.load("train_L_S_smooth.npy")

test_L_S_d = np.load("test_L_S_discrete.npy")
train_L_S_d = np.load("train_L_S_discrete.npy")

test_L_S = np.array(merge(test_L_S_d,test_L_S_s))
train_L_S = np.array(merge(train_L_S_d,train_L_S_s))

LF_l = LF_l + LF_l
print(len(LF_l))

LF_names = [lf.__name__ for lf in LFs] + ['s'+lf.__name__ for lf in LFs]
print(len(LF_names))
print(test_L_S.shape,train_L_S.shape)

16
16
(1872, 2, 16) (3700, 2, 16)


In [18]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from collections import defaultdict
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score

def draw2DArray(a):
    fig = plt.figure(figsize=(6, 3.2))
    ax = fig.add_subplot(111)
    ax.set_title('colorMap')
    plt.imshow(np.array(a))
    ax.set_aspect('equal')
    cax = fig.add_axes([0.12, 0.1, 0.78, 0.8])
    cax.get_xaxis().set_visible(False)
    cax.get_yaxis().set_visible(False)
    cax.patch.set_alpha(0)
    cax.set_frame_on(False)
    plt.colorbar(orientation='vertical')
    plt.show()
    
      
def report2dict(cr):
    # Parse rows
    tmp = list()
    for row in cr.split("\n"):
        parsed_row = [x for x in row.split("  ") if len(x) > 0]
        if len(parsed_row) > 0:
            tmp.append(parsed_row)
    
    # Store in dictionary
    measures = tmp[0]

    D_class_data = defaultdict(dict)
    for row in tmp[1:]:
        class_label = row[0]
        for j, m in enumerate(measures):
            D_class_data[class_label][m.strip()] = float(row[j + 1].strip())
    return pd.DataFrame(D_class_data).T

def predictAndPrint(pl):
    print("acc",accuracy_score(true_labels,pl))
#     print(precision_recall_fscore_support(true_labels,pl,average='macro'))
    print(confusion_matrix(true_labels,pl))
#     draw2DArray(confusion_matrix(gold_labels_dev,pl))
    return report2dict(classification_report(true_labels, pl))# target_names=class_names))
    


def drawPRcurve(y_test,y_score,it_no):
    
    fig = plt.figure()
    splt = fig.add_subplot(111)
    

    precision, recall, _ = precision_recall_curve(y_test, y_score,pos_label=1)

    splt.step(recall, precision, color='b', alpha=0.2,
             where='post')
    splt.fill_between(recall, precision, step='post', alpha=0.2,
                     color='b')
    average_precision = average_precision_score(y_test, y_score)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.05])
    plt.title('{0:d} Precision-Recall curve: AP={1:0.2f}'.format(it_no,
              average_precision))
  

In [275]:
import numpy as np
test_L_S = np.load("test_L_S_discrete.npy")
train_L_S = np.load("train_L_S_discrete.npy")

print(test_L_S.shape,train_L_S.shape)
LF_names= [lf.__name__ for lf in LFs]

(1872, 2, 8) (3700, 2, 8)


In [12]:
import numpy as np
test_L_S = np.load("test_L_S_smooth.npy")
train_L_S = np.load("train_L_S_smooth.npy")

print(test_L_S.shape,train_L_S.shape)

LF_names= ['s'+lf.__name__ for lf in LFs]

(1872, 2, 8) (3700, 2, 8)


In [16]:
#call this only once for a kernel startup
from __future__ import absolute_import, division, print_function

import tensorflow as tf
# BATCH_SIZE = 32
seed = 12

In [13]:
LF_l = [1,1,1,1,-1,-1,-1,-1]
NoOfLFs= len(LF_l)
NoOfClasses = 2
print(len(LF_l))

8


In [313]:
import math
def majority():
    
    # test_L_S_df.mode(axis=1)[0].tolist()
    test_L_S_df = pd.DataFrame((np.array(test_L_S)[:,0,:]))
    # test_L_S_df = test_L_S_df.astype(int)
    test_L_S_df = test_L_S_df.replace(0, np.NaN)

    predicted_labels=test_L_S_df.mode(axis=1)[0].tolist()
    # print(predicted_labels)
    predicted_labels = [ int(x) if not math.isnan(x) else -1 for x in predicted_labels ]
    predicted_labels = [1 if x==1 else 0 for x in predicted_labels ]
    unique, counts = np.unique(predicted_labels, return_counts=True)
    print(dict(zip(unique, counts)))
    print("acc",accuracy_score(true_labels,predicted_labels))
    print(precision_recall_fscore_support(np.array(true_labels),np.array(predicted_labels),average="binary"))


In [297]:
#8 discrete
majority()

{0: 760, 1: 1112}
acc 0.5235042735042735
(0.2068345323741007, 0.9583333333333334, 0.34023668639053256, None)


In [299]:
#8 smooth
majority()

{1: 1872}
acc 0.1282051282051282
(0.1282051282051282, 1.0, 0.22727272727272727, None)


In [314]:
#16 smooth + discrete
majority()

{1: 1872}
acc 0.1282051282051282
(0.1282051282051282, 1.0, 0.22727272727272727, None)


In [14]:

def train(lr,ep,th,af,pcl=np.array([-1,1],dtype=np.float64),norm=True,smooth=True,penalty=0,p3k=3,alp=0.99):
    
    ## lr : learning rate
    ## ep : no of epochs
    ## th : thetas initializer
    ## af : alphas initializer
    ## penalty : {1,2,3} use one of the three penalties, 0: no-penalty
    ## p3k : parameter for penalty-3 
    ## smooth : flag if smooth lfs are used 
    ## make sure smooth/discrete LF data is loaded into train_L_S and test_L_S
    ## pcl : all possible class labels  = [-1,1] for binary, 
    ##       np.arange(0,NoOfClasses) for multiclass
    ## alp : alpha parameter (don't let alpha to be greater than this value)
    BATCH_SIZE = 1
    tf.reset_default_graph()
    

    seed = 12
    with tf.Graph().as_default():

        train_dataset = tf.data.Dataset.from_tensor_slices(train_L_S).batch(BATCH_SIZE)
        dev_dataset = tf.data.Dataset.from_tensor_slices(test_L_S).batch(len(test_L_S))

     
        iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                               train_dataset.output_shapes)
        next_element = iterator.get_next()

        train_init_op = iterator.make_initializer(train_dataset)
        dev_init_op = iterator.make_initializer(dev_dataset)

        next_element = iterator.get_next()
#         print("next_element",next_element)

        alphas = tf.get_variable('alphas', [NoOfLFs],\
                                 initializer=af,\
                                 dtype=tf.float64)

        thetas = tf.get_variable('thetas',[1,NoOfLFs],\
                                initializer=th,\
                        dtype=tf.float64)

#         print("thetas",thetas)
        k = tf.convert_to_tensor(LF_l, dtype=tf.float64)
#         print("k",k)
        l,s =  tf.unstack(next_element,axis=1)
#         print(alphas)
        print(s)
        print("l",l)
#         print(s.graph)
        
        s_ = tf.maximum(tf.subtract(s,tf.minimum(alphas,alp)), 0)
        print("s_",s_)

       
        def iskequalsy(v,s):
            out = tf.where(tf.equal(v,s),tf.ones_like(v),-tf.ones_like(v))
            print("out",out)
            return out

        if(smooth):
            pout = tf.map_fn(lambda c: l*c*s_ ,pcl,name="pout")
        else:
            pout = tf.map_fn(lambda c: l*c ,pcl,name="pout")

        print("pout",pout)    

        t_pout = tf.map_fn(lambda x: tf.matmul(x,thetas,transpose_b=True),pout,\
                           name="t_pout")
    
        print("t_pout",t_pout)

        t =  tf.squeeze(thetas)
        print("t",t)
        
        def ints(y):
            ky = iskequalsy(k,y)
            print("ky",ky)
            out1 = alphas+((tf.exp((t*ky*(1-alphas)))-1)/(t*ky))
            print("intsy",out1)
            return out1
                

        if(smooth):
            #smooth normalizer
            zy = tf.map_fn(lambda y: tf.reduce_prod(1+ints(y),axis=0),\
                           pcl,name="zy")
        else:
            #discrete normalizer
            zy = tf.map_fn(lambda y: tf.reduce_prod(1+tf.exp(t*iskequalsy(k,y)),axis=0),\
                           pcl,name="zy")

    
# 
#         zy = tf.map_fn(lambda y: tf.reduce_prod(1+ints(y),axis=0),\
#                        np.array(NoOfClasses,dtype=np.float64))
        
        print("zy",zy)
        logz = tf.log(tf.reduce_sum(zy,axis=0),name="logz")
        
        print("logz",logz)
        tf.summary.scalar('logz', logz)
        lsp = tf.reduce_logsumexp(t_pout,axis=0)
        print("lsp",lsp)
        tf.summary.scalar('lsp', tf.reduce_sum(lsp))
        
        if(not norm):
            print("unnormlized loss")
            loss = tf.negative(tf.reduce_sum(lsp  ))
        elif(penalty == 1):
            print("penalty1")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                      +tf.reduce_sum(tf.maximum(tf.zeros_like(thetas),-thetas))
        elif(penalty == 2):
            print("penalty2")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                     -tf.minimum( tf.reduce_min(thetas),0.0)
        elif(penalty == 3):
            print("penalty3")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                     +tf.reduce_sum(tf.log(1+tf.exp(-thetas-pk)))
        else:
            print("normalized")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  ))
            
        print("loss",loss)
        tf.summary.scalar('un-normloss', loss)
#         tf.summary.histogram('thetas', t)
#         tf.summary.histogram('alphas', alphas)
#         print("normloss",normloss)
        marginals = tf.nn.softmax(t_pout,axis=0)

        print("marginals",marginals)
        predict = tf.argmax(marginals,axis=0)


    #     pre = tf.metrics.precision(labels,predict)
    #     rec = tf.metrics.recall(labels,predict)
    #     print("loss",loss)
    #     print("nls_",nls_)

    #     global_step = tf.Variable(0, trainable=False,dtype=tf.float64)
    #     starter_learning_rate = 1.0
    #     learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
    #                                            10, 0.96, staircase=True)
    #     train_step = tf.train.AdamOptimizer(learning_rate).minimize(normloss, global_step=global_step) 


    #     train_step = tf.train.AdamOptimizer(0.001).minimize(normloss)
    #     reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    #     reg_constant = 5.0  # Choose an appropriate one.
    #     totalloss = normloss + reg_constant * sum(reg_losses)
        train_step = tf.train.AdamOptimizer(lr).minimize(loss) 
    #     train_step = tf.train.AdagradOptimizer(0.01).minimize(normloss) 
    #     train_step = tf.train.MomentumOptimizer(0.01,0.2).minimize(normloss) 

    #     train_step = tf.train.GradientDescentOptimizer(0.1).minimize(normloss)

        summary_merged = tf.summary.merge_all()
        train_writer = tf.summary.FileWriter('./summary/train',
                                      tf.get_default_graph())
        test_writer = tf.summary.FileWriter('./summary/test')

        init_g = tf.global_variables_initializer()
        init_l = tf.local_variables_initializer()
        with tf.Session() as sess:
            sess.run(init_g)
            sess.run(init_l)

            # Initialize an iterator over the training dataset.
            for en in range(ep):
                sess.run(train_init_op)
                tl = 0
                try:
                    it = 0
                    while True:
                        sm,_,ls,t = sess.run([summary_merged,train_step,loss,thetas])
#                         print(t)
#                         print(tl)
                        train_writer.add_summary(sm, it)
#                         if(ls<1e-5):
#                             break
                        tl = tl + ls
                        it = it + 1
                        
                except tf.errors.OutOfRangeError:
                    pass
                print(en,"loss",tl)

                sess.run(dev_init_op)
                sm,a,t,m,pl = sess.run([summary_merged,alphas,thetas,marginals,predict])
                test_writer.add_summary(sm, en)
                print(a)
                print(t)
                unique, counts = np.unique(pl, return_counts=True)
                print(dict(zip(unique, counts)))
                print("acc",accuracy_score(true_labels,pl))
                print(precision_recall_fscore_support(np.array(true_labels),np.array(pl),average="binary"))
                print()

            # Initialize an iterator over the validation dataset.
            sess.run(dev_init_op)
            a,t,m,pl = sess.run([alphas,thetas,marginals,predict])
            print(a)
            print(t)

            unique, counts = np.unique(pl, return_counts=True)
            print(dict(zip(unique, counts)))

            print("acc",accuracy_score(true_labels,pl))

            predictAndPrint(pl)
            print(precision_recall_fscore_support(np.array(true_labels),np.array(pl),average="binary"))

#             cf = confusion_matrix(true_labels,pl)
#             print(cf)
    return pl

In [259]:
# 5 LFs
train(0.001/len(train_L_S),5,th = tf.truncated_normal_initializer(1,0.1,seed),\
                            af = tf.truncated_normal_initializer(0.9,0.001,seed),
                          pcl=np.array([-1,1],dtype=np.float64),smooth=False,penalty=2)

Tensor("unstack:1", shape=(?, 5), dtype=float64)
l Tensor("unstack:0", shape=(?, 5), dtype=float64)
s_ Tensor("Maximum:0", shape=(?, 5), dtype=float64)
pout Tensor("pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 5), dtype=float64)
t_pout Tensor("t_pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t Tensor("Squeeze:0", shape=(5,), dtype=float64)
out Tensor("zy/while/Select:0", shape=(5,), dtype=float64)
zy Tensor("zy/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("logz:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(?, 1), dtype=float64)
penalty2
loss Tensor("sub_1:0", shape=(), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
0 loss 13726.844830684313
[0.90117545 0.89814215 0.9000956  0.90000624 0.90106378]
[[1.1169368  0.81415761 1.00906606 1.00020806 1.10542386]]
{0: 856, 1: 1016}
acc 0.5331196581196581
(0.18799212598425197, 0.7958333333333333, 0.3041401273

array([[0],
       [0],
       [1],
       ...,
       [1],
       [0],
       [1]])

In [224]:
# 8 LFs
train(0.001/len(train_L_S),5,th = tf.truncated_normal_initializer(1,0.1,seed),\
                            af = tf.truncated_normal_initializer(0.9,0.001,seed),
                          pcl=np.array([-1,1],dtype=np.float64),smooth=False,penalty=2)

Tensor("unstack:1", shape=(?, 8), dtype=float64)
l Tensor("unstack:0", shape=(?, 8), dtype=float64)
s_ Tensor("Maximum:0", shape=(?, 8), dtype=float64)
pout Tensor("pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 8), dtype=float64)
t_pout Tensor("t_pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t Tensor("Squeeze:0", shape=(8,), dtype=float64)
out Tensor("zy/while/Select:0", shape=(8,), dtype=float64)
zy Tensor("zy/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("logz:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(?, 1), dtype=float64)
penalty2
loss Tensor("sub_1:0", shape=(), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
0 loss 22236.173218776625
[0.90117545 0.89814215 0.9000956  0.90000624 0.90106378 0.90041339
 0.90169206 0.90031208]
[[1.11758662 0.81475495 1.00975357 1.00072731 1.10540322 1.04034068
  1.16822805 1.03020839]]
{0: 760, 1: 1112}
acc 0.52

array([[0],
       [0],
       [1],
       ...,
       [1],
       [0],
       [1]])

In [241]:
#8
for i in np.linspace(0,1,11):
    print("alpha-mean",i)
    train(0.001/len(train_L_S),5,th = tf.truncated_normal_initializer(1,0.1,seed),\
                                af = tf.truncated_normal_initializer(i,0.001,seed),
                              pcl=np.array([-1,1],dtype=np.float64),smooth=True,penalty=0)

alpha-mean 0.0
Tensor("unstack:1", shape=(?, 8), dtype=float64)
l Tensor("unstack:0", shape=(?, 8), dtype=float64)
s_ Tensor("Maximum:0", shape=(?, 8), dtype=float64)
pout Tensor("pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 8), dtype=float64)
t_pout Tensor("t_pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t Tensor("Squeeze:0", shape=(8,), dtype=float64)
out Tensor("zy/while/Select:0", shape=(8,), dtype=float64)
ky Tensor("zy/while/Select:0", shape=(8,), dtype=float64)
intsy Tensor("zy/while/add:0", shape=(8,), dtype=float64)
zy Tensor("zy/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("logz:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(?, 1), dtype=float64)
normalized
loss Tensor("Neg:0", shape=(), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
0 loss 18360.82658841929
[ 1.76799135e-04 -2.85665373e-03 -9.03101723e-04 -9.92460379e-04
  6.45089183e-05

/home/vinay/snorkelEnv/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0 loss 18936.46658980817
[0.10017841 0.09714484 0.09909847 0.09900911 0.10006657 0.10139972
 0.10268999 0.10130995]
[[1.11852808 0.81520207 1.01054455 1.00161446 1.10736016 1.04036184
  1.16821422 1.03021626]]
{1: 1872}
acc 0.1282051282051282
(0.1282051282051282, 1.0, 0.22727272727272727, None)

1 loss 18897.673256851478
[0.09918023 0.09614639 0.09810019 0.09801082 0.09906752 0.10237525
 0.1036886  0.1023085 ]
[[1.11951404 0.81619089 1.01153342 1.00260478 1.10834631 1.03940143
  1.16722211 1.02922432]]
{1: 1872}
acc 0.1282051282051282
(0.1282051282051282, 1.0, 0.22727272727272727, None)

2 loss 18858.81664842444
[0.09818195 0.09514786 0.09710183 0.09701245 0.09806831 0.10334948
 0.10468725 0.10330709]
[[1.12050014 0.81717983 1.01252243 1.00359523 1.10933272 1.03844286
  1.16623005 1.02823244]]
{1: 1872}
acc 0.1282051282051282
(0.1282051282051282, 1.0, 0.22727272727272727, None)

3 loss 18819.90564977269
[0.09718361 0.09414927 0.09610341 0.09601402 0.097069   0.10432253
 0.10568593 0.10

In [20]:
#8
import numpy as np
for i in np.linspace(0.5,0.6,11):
    print("alpha-mean",i)
    train(0.001/len(train_L_S),5,th = tf.truncated_normal_initializer(1,0.1,seed),\
                                af = tf.truncated_normal_initializer(i,0.001,seed),
                              pcl=np.array([-1,1],dtype=np.float64),smooth=True,penalty=0)

alpha-mean 0.5
Tensor("unstack:1", shape=(?, 8), dtype=float64)
l Tensor("unstack:0", shape=(?, 8), dtype=float64)
s_ Tensor("Maximum:0", shape=(?, 8), dtype=float64)
pout Tensor("pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 8), dtype=float64)
t_pout Tensor("t_pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t Tensor("Squeeze:0", shape=(8,), dtype=float64)
out Tensor("zy/while/Select:0", shape=(8,), dtype=float64)
ky Tensor("zy/while/Select:0", shape=(8,), dtype=float64)
intsy Tensor("zy/while/add:0", shape=(8,), dtype=float64)
zy Tensor("zy/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("logz:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(?, 1), dtype=float64)
normalized
loss Tensor("Neg:0", shape=(), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
0 loss 20583.92165970571
[0.50045124 0.49738986 0.49935774 0.49926707 0.50036045 0.50140421
 0.50253896 0.

In [21]:
#8
import numpy as np
for i in np.linspace(0.6,0.7,11):
    print("alpha-mean",i)
    train(0.001/len(train_L_S),5,th = tf.truncated_normal_initializer(1,0.1,seed),\
                                af = tf.truncated_normal_initializer(i,0.001,seed),
                              pcl=np.array([-1,1],dtype=np.float64),smooth=True,penalty=0)

alpha-mean 0.6
Tensor("unstack:1", shape=(?, 8), dtype=float64)
l Tensor("unstack:0", shape=(?, 8), dtype=float64)
s_ Tensor("Maximum:0", shape=(?, 8), dtype=float64)
pout Tensor("pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 8), dtype=float64)
t_pout Tensor("t_pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t Tensor("Squeeze:0", shape=(8,), dtype=float64)
out Tensor("zy/while/Select:0", shape=(8,), dtype=float64)
ky Tensor("zy/while/Select:0", shape=(8,), dtype=float64)
intsy Tensor("zy/while/add:0", shape=(8,), dtype=float64)
zy Tensor("zy/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("logz:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(?, 1), dtype=float64)
normalized
loss Tensor("Neg:0", shape=(), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
0 loss 20580.630500302086
[0.60097827 0.59790639 0.5998904  0.60053532 0.60089577 0.60139461
 0.60218203 0

In [322]:
#8
for i in np.linspace(0,1,11):
    print("alpha-mean",i)
    train(0.001/len(train_L_S),5,th = tf.truncated_normal_initializer(1,0.1,seed),\
                                af = tf.truncated_normal_initializer(i,0.001,seed),
                              pcl=np.array([-1,1],dtype=np.float64),smooth=True,penalty=2)

alpha-mean 0.0
Tensor("unstack:1", shape=(?, 8), dtype=float64)
l Tensor("unstack:0", shape=(?, 8), dtype=float64)
s_ Tensor("Maximum:0", shape=(?, 8), dtype=float64)
pout Tensor("pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 8), dtype=float64)
t_pout Tensor("t_pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t Tensor("Squeeze:0", shape=(8,), dtype=float64)
out Tensor("zy/while/Select:0", shape=(8,), dtype=float64)
ky Tensor("zy/while/Select:0", shape=(8,), dtype=float64)
intsy Tensor("zy/while/add:0", shape=(8,), dtype=float64)
zy Tensor("zy/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("logz:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(?, 1), dtype=float64)
penalty2
loss Tensor("sub_1:0", shape=(), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
0 loss 18360.82658841929
[ 1.76799135e-04 -2.85665373e-03 -9.03101723e-04 -9.92460379e-04
  6.45089183e-05

/home/vinay/snorkelEnv/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
0 loss 18936.46658980817
[0.10017841 0.09714484 0.09909847 0.09900911 0.10006657 0.10139972
 0.10268999 0.10130995]
[[1.11852808 0.81520207 1.01054455 1.00161446 1.10736016 1.04036184
  1.16821422 1.03021626]]
{1: 1872}
acc 0.1282051282051282
(0.1282051282051282, 1.0, 0.22727272727272727, None)

1 loss 18897.673256851478
[0.09918023 0.09614639 0.09810019 0.09801082 0.09906752 0.10237525
 0.1036886  0.1023085 ]
[[1.11951404 0.81619089 1.01153342 1.00260478 1.10834631 1.03940143
  1.16722211 1.02922432]]
{1: 1872}
acc 0.1282051282051282
(0.1282051282051282, 1.0, 0.22727272727272727, None)

2 loss 18858.81664842444
[0.09818195 0.09514786 0.09710183 0.09701245 0.09806831 0.10334948
 0.10468725 0.10330709]
[[1.12050014 0.81717983 1.01252243 1.00359523 1.10933272 1.03844286
  1.16623005 1.02823244]]
{1: 1872}
acc 0.1282051282051282
(0.1282051282051282, 1.0, 0.22727272727272727, None)

3 loss 18819.90564977269
[0.09718361 0.094

In [315]:
#8
for i in np.linspace(0,1,11):
    print("alpha-mean",i)
    train(0.001/len(train_L_S),5,th = tf.truncated_normal_initializer(1,0.1,seed),\
                                af = tf.truncated_normal_initializer(i,0.001,seed),
                              pcl=np.array([-1,1],dtype=np.float64),smooth=True,penalty=2)

alpha-mean 0.0
Tensor("unstack:1", shape=(?, 16), dtype=float64)
l Tensor("unstack:0", shape=(?, 16), dtype=float64)
s_ Tensor("Maximum:0", shape=(?, 16), dtype=float64)
pout Tensor("pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 16), dtype=float64)
t_pout Tensor("t_pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t Tensor("Squeeze:0", shape=(16,), dtype=float64)
out Tensor("zy/while/Select:0", shape=(16,), dtype=float64)
ky Tensor("zy/while/Select:0", shape=(16,), dtype=float64)
intsy Tensor("zy/while/add:0", shape=(16,), dtype=float64)
zy Tensor("zy/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("logz:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(?, 1), dtype=float64)
penalty2
loss Tensor("sub_1:0", shape=(), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
0 loss 36272.053384859966
[ 0.00096819 -0.00246081 -0.00022373 -0.00023594  0.00193544  0.00139114

/home/vinay/snorkelEnv/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0 loss 37001.80337293428
[0.1009594  0.09753896 0.09977131 0.09975826 0.10190758 0.10138807
 0.10254439 0.10129883 0.10046577 0.0996496  0.09883802 0.09966851
 0.09882508 0.10176176 0.1014063  0.10142053]
[[1.11778627 0.81482251 1.00990119 1.00089162 1.10560216 1.04037828
  1.16841878 1.03022762 1.14731924 1.06571036 0.98455647 1.06760411
  0.98336709 1.07761919 1.03982311 1.04124675]]
{1: 1872}
acc 0.1282051282051282
(0.1282051282051282, 1.0, 0.22727272727272727, None)

1 loss 36954.36754660866
[0.10075163 0.09693414 0.09945476 0.09951616 0.10275094 0.1023414
 0.10337612 0.10225939 0.09946814 0.0986519  0.09784025 0.09867081
 0.09782537 0.10271729 0.10240432 0.10241855]
[[1.11802361 0.81543403 1.01023844 1.00115724 1.10482484 1.03944844
  1.16765413 1.02928613 1.14830518 1.06669767 0.98554495 1.06859322
  0.98435514 1.0766858  1.03883422 1.04025786]]
{1: 1872}
acc 0.1282051282051282
(0.1282051282051282, 1.0, 0.22727272727272727, None)

2 loss 36907.17321961758
[0.10055253 0.09633298 0

In [273]:
#16
for i in np.linspace(0,1,11):
    print("alpha-mean",i)
    train(0.001/len(train_L_S),5,th = tf.truncated_normal_initializer(1,0.1,seed),\
                                af = tf.truncated_normal_initializer(i,0.001,seed),
                              pcl=np.array([-1,1],dtype=np.float64),smooth=True,penalty=0)

alpha-mean 0.0
Tensor("unstack:1", shape=(?, 16), dtype=float64)
l Tensor("unstack:0", shape=(?, 16), dtype=float64)
s_ Tensor("Maximum:0", shape=(?, 16), dtype=float64)
pout Tensor("pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 16), dtype=float64)
t_pout Tensor("t_pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t Tensor("Squeeze:0", shape=(16,), dtype=float64)
out Tensor("zy/while/Select:0", shape=(16,), dtype=float64)
ky Tensor("zy/while/Select:0", shape=(16,), dtype=float64)
intsy Tensor("zy/while/add:0", shape=(16,), dtype=float64)
zy Tensor("zy/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("logz:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(?, 1), dtype=float64)
normalized
loss Tensor("Neg:0", shape=(), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
0 loss 36272.053384859966
[ 0.00096819 -0.00246081 -0.00022373 -0.00023594  0.00193544  0.00139114

/home/vinay/snorkelEnv/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0 loss 37001.80337293428
[0.1009594  0.09753896 0.09977131 0.09975826 0.10190758 0.10138807
 0.10254439 0.10129883 0.10046577 0.0996496  0.09883802 0.09966851
 0.09882508 0.10176176 0.1014063  0.10142053]
[[1.11778627 0.81482251 1.00990119 1.00089162 1.10560216 1.04037828
  1.16841878 1.03022762 1.14731924 1.06571036 0.98455647 1.06760411
  0.98336709 1.07761919 1.03982311 1.04124675]]
{1: 1872}
acc 0.1282051282051282
(0.1282051282051282, 1.0, 0.22727272727272727, None)

1 loss 36954.36754660866
[0.10075163 0.09693414 0.09945476 0.09951616 0.10275094 0.1023414
 0.10337612 0.10225939 0.09946814 0.0986519  0.09784025 0.09867081
 0.09782537 0.10271729 0.10240432 0.10241855]
[[1.11802361 0.81543403 1.01023844 1.00115724 1.10482484 1.03944844
  1.16765413 1.02928613 1.14830518 1.06669767 0.98554495 1.06859322
  0.98435514 1.0766858  1.03883422 1.04025786]]
{1: 1872}
acc 0.1282051282051282
(0.1282051282051282, 1.0, 0.22727272727272727, None)

2 loss 36907.17321961758
[0.10055253 0.09633298 0

In [274]:
#16 LFs
for i in np.linspace(0,1,11):
    print("alpha-mean",i)
    train(0.001/len(train_L_S),5,th = tf.truncated_normal_initializer(1,0.1,seed),\
                                af = tf.truncated_normal_initializer(i,0.001,seed),
                              pcl=np.array([-1,1],dtype=np.float64),norm=False,smooth=True,penalty=0)

alpha-mean 0.0
Tensor("unstack:1", shape=(?, 16), dtype=float64)
l Tensor("unstack:0", shape=(?, 16), dtype=float64)
s_ Tensor("Maximum:0", shape=(?, 16), dtype=float64)
pout Tensor("pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 16), dtype=float64)
t_pout Tensor("t_pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t Tensor("Squeeze:0", shape=(16,), dtype=float64)
out Tensor("zy/while/Select:0", shape=(16,), dtype=float64)
ky Tensor("zy/while/Select:0", shape=(16,), dtype=float64)
intsy Tensor("zy/while/add:0", shape=(16,), dtype=float64)
zy Tensor("zy/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("logz:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(?, 1), dtype=float64)
unnormlized loss
loss Tensor("Neg:0", shape=(), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
0 loss -10687.358578682832
[ 0.00074551 -0.00252045 -0.00038564 -0.00042053  0.00115683  0.0

/home/vinay/snorkelEnv/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0 loss -9202.01628208825
[0.10074565 0.09747973 0.09961456 0.09957948 0.10115641 0.10045837
 0.10180204 0.10034464 0.10046471 0.09964859 0.09883705 0.0996675
 0.09882516 0.1008308  0.10140647 0.10142071]
[[1.11797443 0.81487694 1.01004069 1.00105107 1.10628524 1.04129357
  1.16909601 1.03117529 1.14732285 1.06571368 0.98455957 1.06760814
  0.98337033 1.07853661 1.03982545 1.0412491 ]]
{1: 1872}
acc 0.1282051282051282
(0.1282051282051282, 1.0, 0.22727272727272727, None)

1 loss -9248.402994353539
[0.10031883 0.09681137 0.09913554 0.09914966 0.10125319 0.10050171
 0.10191167 0.10037879 0.09946562 0.09864948 0.09783792 0.09866839
 0.09782604 0.10087414 0.1024051  0.10241933]
[[1.11840126 0.81554529 1.01051972 1.00148089 1.10618846 1.04125023
  1.16898638 1.03114113 1.14831256 1.0667046  0.98555118 1.06860205
  0.98436095 1.07849327 1.03883865 1.0402623 ]]
{1: 1872}
acc 0.1282051282051282
(0.1282051282051282, 1.0, 0.22727272727272727, None)

2 loss -9294.860106072198
[0.09989205 0.09614295

In [261]:
import scipy.sparse as sp
import _pickle as pkl
# L_train = pkl.load(open("train_L_S_discrete.p","rb"))
# L_train = sp.csr_matrix(L_train)

# L_gold = pkl.load(open("gold_discrete.p","rb"))
# print(np.array(L_gold).shape)
# L_gold = sp.csr_matrix(L_gold)

L_train = np.load("train_L_S_discrete.npy")
L_train = L_train[:,0,:].astype(int)
print(np.array(L_train).shape)
L_train = sp.csr_matrix(L_train)

L_gold = np.load("test_L_S_discrete.npy")
L_gold = L_gold[:,0,:].astype(int)
print(np.array(L_gold).shape)
L_gold = sp.csr_matrix(L_gold)

from snorkel.learning import GenerativeModel
import time
import datetime

gen_model = GenerativeModel()

start_time = time.time()

lt = time.localtime()

print("started at: {}-{}-{}, {}:{}:{}".format(lt.tm_mday,lt.tm_mon,lt.tm_year,lt.tm_hour,lt.tm_min,lt.tm_sec))

gen_model.train(L_train, epochs = 100, cardinality=2)
# gen_model.train(L_train, epochs=100, decay=0.95, step_size=0.1 / L_train.shape[0], reg_param=1e-6)


print("trained in ",str(datetime.timedelta(seconds=time.time() - start_time)))

(3700, 5)
(1872, 5)
started at: 9-6-2018, 20:22:49
trained in  0:00:06.133447


In [262]:
# 5 LFs
import numpy as np
dev_marginals = gen_model.marginals(L_gold)
dev_marginals = np.array(dev_marginals)
print(dev_marginals.shape)

# GenLabels = np.argmax(dev_marginals,axis=1)
GenLabels =  np.array([1 if m > 0.5 else 0 for m in dev_marginals])
print(GenLabels.shape)

print(precision_recall_fscore_support(np.array(true_labels),GenLabels,average="binary"))


(1872,)
(1872,)
(0.18799212598425197, 0.7958333333333333, 0.3041401273885351, None)


In [219]:
# 8 discrete LFs
import numpy as np
dev_marginals = gen_model.marginals(L_gold)
dev_marginals = np.array(dev_marginals)
print(dev_marginals.shape)

# GenLabels = np.argmax(dev_marginals,axis=1)
GenLabels =  np.array([1 if m > 0.5 else 0 for m in dev_marginals])
print(GenLabels.shape)

print(precision_recall_fscore_support(np.array(true_labels),GenLabels,average="binary"))


(1872,)
(1872,)
(0.20664869721473494, 0.9583333333333334, 0.33998521803399856, None)
